In [1]:
#!pip install instructor
#!pip install openai
#!pip install pydantic
#!pip install jsonref
#!pip install langchain
#!pip install -U swifter

In [2]:
import instructor
from pydantic import BaseModel
from openai import OpenAI
from langchain.prompts import PromptTemplate
from tqdm import tqdm
import os 

class Evaluation(BaseModel):
    fluency: int
    adequacy: int

api_key = os.getenv("OPENAI_API_KEY")
client = instructor.from_openai(OpenAI(api_key =api_key))

prompt = """ You are a helpful language evaluator who can evaluate
input sentence2 and provide an evaluation of its fluency with a
likert scale rating of 1-5, 5 being highly fluent.
You will also have to compate two sentences and judge how adequate
is input sentence 2 with respect to input sentence 1, again with a likert scale rating of 1-5, 5 being highly adequate.
Here are the sentences: input_sentence1: {input_sentence1}, input_sentence2:{input_sentence2}"""

template = PromptTemplate(
    input_variables=["input_sentence1", "input_sentence2"],
    template=prompt,
)

def eval_single(row):
  i, r = row
  input_sentence1, input_sentence2 = r["Expected Caption"], r["Generated Caption"]
  final_prompt = template.format(input_sentence1=input_sentence1, input_sentence2=input_sentence2)
  eval_info = client.chat.completions.create(
    model="gpt-4o",
    response_model=Evaluation,
    messages=[{"role": "user", "content": final_prompt}],
  )
  #print (eval_info.model_dump())
    
  return eval_info.model_dump()



In [3]:
import pandas as pd
import json
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to be applied to each element in the list

def get_results_for_single_file(results_dir,file):
    file_path = os.path.join(results_dir,file)
    df = pd.read_csv(file_path)
    num_threads = 8

    # Create a ThreadPoolExecutor
    results = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Submit tasks to the thread pool
        futures = [executor.submit(eval_single, item) for item in df.iterrows()]

        # Process the results as they complete
        for future in tqdm(as_completed(futures)):
            results.append(future.result())

    df["eval"] = pd.Series(results)
    df.to_csv(file.replace("csv","eval.csv"))
    average_adequacy = df["eval"].apply(lambda x: x["adequacy"]).mean()
    average_fluency = df["eval"].apply(lambda x: x["fluency"]).mean()
    return {"Avg. Fluency":average_fluency, "Avg. Adequacy": average_adequacy}

In [4]:
import os
results_dir = "../results"
all_res = {}
for file in os.listdir(results_dir):
    results = get_results_for_single_file(results_dir, file)
    print(results)
    all_res[file.replace("csv","")] = results

results_df = pd.DataFrame(all_res).transpose()
results_df.to_csv("all_gpt4_results.csv")

161it [00:22,  7.21it/s]


{'Avg. Fluency': 4.6645962732919255, 'Avg. Adequacy': 1.031055900621118}


1987it [03:32,  9.36it/s]


{'Avg. Fluency': 4.46250629089079, 'Avg. Adequacy': 1.516859587317564}


162it [00:22,  7.36it/s]


...

{'Avg. Fluency': 4.628048780487805, 'Avg. Adequacy': 1.024390243902439}


1987it [03:55,  8.45it/s]


{'Avg. Fluency': 4.732259687971816, 'Avg. Adequacy': 1.0452944136889784}


161it [00:19,  8.16it/s]

{'Avg. Fluency': 4.53416149068323, 'Avg. Adequacy': 1.2422360248447204}
